In [276]:
import cv2

# imagePath = 'C:/Users/Nolan/Documents/Python Scripts/npm/analysis/data/calibration 20x.png'
# imageName = imagePath.split(r'/')[-1].split('.')[0]
# image = cv2.imread(imagePath)

# if image is None:
#     raise ValueError("Could not load image. Check the file path.")

def scaleBar(image, scaleFactor=6.9, scaleLength = 30, scaleUnit = 'um', barHeight = 10, border = 1, divisions = 30, fontScale = 1, thickness = 1, posX = 20, posY = 30):
    scaleLengthPixels = int(scaleLength*scaleFactor)
    shape = image.shape
    if len(shape) == 2:  # Grayscale image
        height, width = shape
        channels = 1  # Grayscale has one channel
    elif len(shape) == 3:  # Color image
        height, width, channels = shape
    else:
        raise ValueError("Unexpected image shape.")
    
    scaleBarStart = (posX+border, height - posY)
    scaleBarEnd = (scaleBarStart[0] + scaleLengthPixels, scaleBarStart[1]-barHeight)
    
    def rectBorder(start, end, borderL, borderR, borderU, borderD): # return border positions for any given rect
        newStart = (start[0]-borderL, start[1]-borderU)
        newEnd = (end[0] + borderR, end[1]+borderD)
        return [newStart, newEnd]
    
    textPosition = (scaleBarStart[0], height - 40 - barHeight)  # Adjust position as needed
    
    (textWidth, textHeight), _ = cv2.getTextSize(f'{scaleLength} {scaleUnit}', cv2.FONT_HERSHEY_DUPLEX, fontScale, thickness)
    textStart = (textPosition[0], textPosition[1] - textHeight)
    textEnd = (textPosition[0] + textWidth, textPosition[1])
    
    scaleBarBackground = rectBorder(textStart, textEnd, 0, 0, 5, 5)
    
    cv2.rectangle(image, scaleBarBackground[0], scaleBarBackground[1], color=(255, 255, 255), thickness=-1) # background
    
    i=0
    while True: # black/white divisions
        dist = int((scaleBarEnd[0] - scaleBarStart[0])/divisions)
        start = (scaleBarStart[0]+i*dist,
                 scaleBarStart[1])
        end = (scaleBarStart[0]+(i+1)*dist,
               scaleBarEnd[1])
        print(start[0], end[0], dist, i)
        if end[0]+(i+1)*dist == scaleBarEnd[0] or end[0] >= scaleBarEnd[0]: # stop if end of bar reached
            end = scaleBarEnd
            if i % 2 == 0:
                cv2.rectangle(image, start, end, color=(0, 0, 0), thickness=-1)
            else:
                cv2.rectangle(image, start, end, color=(255, 255, 255), thickness=-1)
            break
        elif i % 2 == 0:
            cv2.rectangle(image, start, end, color=(0, 0, 0), thickness=-1)
        else:
            cv2.rectangle(image, start, end, color=(255, 255, 255), thickness=-1)
        i+=1
    
    cv2.rectangle(image, scaleBarStart, scaleBarEnd, color=(0, 0, 0), thickness=border) # scale bar border
    
    cv2.putText(image, f'{scaleLength} {scaleUnit}', textPosition, cv2.FONT_HERSHEY_DUPLEX, # text
                fontScale=fontScale, color=(0, 0, 0), thickness=thickness, lineType=cv2.LINE_AA)

21 34 13 0
34 47 13 1
47 60 13 2
60 73 13 3
73 86 13 4
86 99 13 5
99 112 13 6
112 125 13 7
125 138 13 8
138 151 13 9
151 164 13 10


In [282]:
cap = cv2.VideoCapture('video1.avi')
ret, frame = cap.read()
# get the video frame height and width
height, width, channel = frame.shape
save_name = 'demonstration.mp4'
# define codec and create VideoWriter object
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*'mp4v'), 30, 
    (width, height)
)
            
frame_count = 0

while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        orig_frame = frame.copy()
        scaleBar(orig_frame, scaleFactor=12.6, scaleLength=20, divisions = 10)
        cv2.imshow('New Video', orig_frame)
        out.write(orig_frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 25 3
121 146 25 4
146 171 25 5
171 196 25 6
196 221 25 7
221 246 25 8
246 271 25 9
271 296 25 10
21 46 25 0
46 71 25 1
71 96 25 2
96 121 2